In [183]:
import numpy as np
import pandas as pd

In [184]:
taux_inflation = 1 + 10/100
taux_revalorisation_bareme = 1 + 5/100

In [185]:
taux = np.array([0, 0.1, 0.2, 0.5, 0.75])
seuils_tranches_initiales = np.array([0, 20000, 30000, 40000, 50000, np.inf])

def impots(revenu, seuils_tranches, taux):
    assert len(seuils_tranches) == len(taux) + 1
    return np.sum(np.diff(np.minimum(np.array([revenu]), seuils_tranches))  * taux)

In [186]:
def calcule_vecteur_inflateur(nombre_annees, taux):
    return  np.cumprod([1] + [taux for _ in range(nombre_annees - 1)])

In [187]:
nombre_annees = 10
annees = range(nombre_annees)

inflateur_prix = calcule_vecteur_inflateur(nombre_annees, taux_inflation)
inflateur_baremes = calcule_vecteur_inflateur(nombre_annees, taux_revalorisation_bareme)

seuils_par_annee = {
    annee: seuils_tranches_initiales * inflateur_baremes[annee]
    for annee in annees
    }

impots_par_annee = {
    annee: lambda revenu: impots(revenu, seuils_par_annee[annee], taux)
    for annee in annees
    }

In [194]:
revenus_euros_constants = np.concatenate([2*[0], np.linspace(40000, 100000, 5), (nombre_annees - 5 - 2)* [0]])
revenus_euros_constants

array([     0.,      0.,  40000.,  55000.,  70000.,  85000., 100000.,
            0.,      0.,      0.])

In [195]:
revenus_euros_courants = revenus_euros_constants * inflateur_prix
revenus_euros_courants

array([     0.  ,      0.  ,  48400.  ,  73205.  , 102487.  , 136893.35,
       177156.1 ,      0.  ,      0.  ,      0.  ])

In [196]:
revenus_passes = np.concatenate([revenus_euros_courants[1:], [0]])
revenus_passes

array([     0.  ,  48400.  ,  73205.  , 102487.  , 136893.35, 177156.1 ,
            0.  ,      0.  ,      0.  ,      0.  ])

In [197]:
def calcul_impots(impots_par_annee, revenu):
    ensemble_des_impots = [
        impots_par_annee[annee](revenu[annee])
        for annee in set(impots_par_annee)
        ]
    return ensemble_des_impots

In [198]:
calcul_impots(impots_par_annee, revenus_euros_courants)  

[0.0,
 0.0,
 1923.3589201074205,
 10229.920328365239,
 31101.067628633806,
 56905.830128633825,
 87102.89262863384,
 0.0,
 0.0,
 0.0]

In [199]:
calcul_impots(impots_par_annee, revenus_passes)

[0.0,
 1923.3589201074205,
 10229.920328365239,
 31101.067628633806,
 56905.830128633825,
 87102.89262863384,
 0.0,
 0.0,
 0.0,
 0.0]